In [14]:
import numpy as np

a1 = np.array(([1],[0]))
a2 = np.array(([1],[0]))
np.zeros([2, 2], 'float64')[a1,a2]

array([[ 0.],
       [ 0.]])

In [15]:
a1[-1]

array([0])

In [176]:
import theano
from theano import tensor as T



W_values = np.random.randn(3,3)
bvis_values = np.random.randn(3)
bhid_values = np.random.randn(3)
W = theano.shared(W_values) # we assume that ``W_values`` contains the
                            # initial values of your weight matrix

bvis = theano.shared(bvis_values)
bhid = theano.shared(bhid_values)

trng = T.shared_randomstreams.RandomStreams(1234)

def OneStep(vsample) :
    hmean = T.nnet.sigmoid(theano.dot(vsample, W) + bhid)
    hsample = trng.binomial(size=hmean.shape, n=1, p=hmean)
    vmean = T.nnet.sigmoid(theano.dot(hsample, W.T) + bvis)
    return trng.binomial(size=vsample.shape, n=1, p=vmean,
                         dtype=theano.config.floatX)*5

sample = theano.tensor.matrix()
input = sample[:,:-2].flatten()

values, updates = theano.scan(OneStep, outputs_info=input, n_steps=10)

gibbs10 = theano.function([sample], values[-1], updates=updates)

In [173]:
a = np.array([[1,2,3],[1,2,3],[1,2,3]], dtype = theano.config.floatX)

In [174]:
gibbs10(a)

array([ 0.,  0.,  5.], dtype=float32)

In [152]:
a[:,:-2].flatten()

array([ 1.,  1.,  1.], dtype=float32)

In [156]:
a.sum(axis=1)

array([ 6.,  6.,  6.], dtype=float32)

In [609]:
def MultSim( v, v_sum) :
    return trng.multinomial(size=None, 
                            n=v_sum, 
                            pvals=v/v_sum[:,None], 
                            dtype=theano.config.floatX)

input = T.matrix()
input_rSum = T.vector()

values, updates = theano.scan(MultSim, outputs_info=input, non_sequences = input_rSum, n_steps=3)

gibbs_mult = theano.function([input,input_rSum], values, updates=updates)



In [610]:
b = np.array([[2,0,0],[0,0,2],[1,1,1]], dtype = theano.config.floatX)
out = gibbs_mult(b,b.sum(axis=1))

In [670]:
print(out.shape)
out


(3, 3, 3)


array([[[ 2.,  0.,  0.],
        [ 0.,  0.,  2.],
        [ 1.,  1.,  1.]],

       [[ 2.,  0.,  0.],
        [ 0.,  0.,  2.],
        [ 0.,  1.,  2.]],

       [[ 2.,  0.,  0.],
        [ 0.,  0.,  2.],
        [ 0.,  1.,  2.]]], dtype=float32)